<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [4]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [5]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [6]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


In [8]:
import requests

static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Page fetched successfully!")
else:
    print(f"Failed to fetch page. Status code: {response.status_code}")


Page fetched successfully!


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [9]:
import requests

# Static Wikipedia URL for Falcon 9 and Falcon Heavy launches
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the GET request
response = requests.get(static_url)

# Check the response status
if response.status_code == 200:
    print("✅ Page fetched successfully!")
    html_content = response.text  # This contains the full HTML of the page
else:
    print(f"❌ Failed to fetch page. Status code: {response.status_code}")


✅ Page fetched successfully!


Create a `BeautifulSoup` object from the HTML `response`


In [10]:
from bs4 import BeautifulSoup

# Assuming html_content is the response text from the GET request
soup = BeautifulSoup(html_content, "html.parser")

# Example: Print the title of the page
print(soup.title.string)



List of Falcon 9 and Falcon Heavy launches - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [11]:
# Use soup.title attribute
# Print the page title to verify parsing
print(soup.title.string)


List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [13]:
from bs4 import BeautifulSoup
import requests

# Fetch the HTML content
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(url)
html_content = response.text

# Parse the HTML
soup = BeautifulSoup(html_content, "html.parser")

# Find all tables
tables = soup.find_all("table")

# Print how many tables were found
print(f"🔍 Found {len(tables)} tables on the page.")

# Optional: Preview the first table
print(tables[0].prettify()[:1000])  # Print first 1000 characters of the first table


🔍 Found 25 tables on the page.
<table class="col-begin" role="presentation">
 <tbody>
  <tr>
   <td class="col-break">
    <div class="mw-heading mw-heading3">
     <h3 id="Rocket_configurations">
      Rocket configurations
     </h3>
    </div>
    <div class="chart noresize" style="padding-top:10px;margin-top:1em;max-width:420px;">
     <div style="position:relative;min-height:320px;min-width:420px;max-width:420px;">
      <div style="float:right;position:relative;min-height:240px;min-width:320px;max-width:320px;border-left:1px black solid;border-bottom:1px black solid;">
       <div style="position:absolute;left:3px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid LightSteelBlue;border-bottom:none;overflow:hidden;" title="[[Falcon 9 v1.0]]: 2">
       </div>
       <div style="position:absolute;left:55px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-pri

In [14]:
from bs4 import BeautifulSoup
import requests

# Fetch the HTML content
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find all tables
tables = soup.find_all("table")

# Loop through each table and extract column headers
for idx, table in enumerate(tables):
    header_row = table.find("tr")
    if header_row:
        headers = [th.get_text(strip=True) for th in header_row.find_all("th")]
        print(f"🧾 Table {idx + 1} column headers:")
        print(headers)
        print("-" * 50)


🧾 Table 1 column headers:
[]
--------------------------------------------------
🧾 Table 2 column headers:
[]
--------------------------------------------------
🧾 Table 3 column headers:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']
--------------------------------------------------
🧾 Table 4 column headers:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']
--------------------------------------------------
🧾 Table 5 column headers:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']
--------------------------------------------------
🧾 Table 6 column headers:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass',

Starting from the third table is our target table contains the actual launch records.


In [15]:
from bs4 import BeautifulSoup
import requests

# Fetch the HTML content
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find all tables
tables = soup.find_all("table")

# Target the third table (index 2 since Python is zero-based)
target_table = tables[2]

# Extract column headers from the first row
header_row = target_table.find("tr")
column_names = [th.get_text(strip=True) for th in header_row.find_all("th")]

print("🚀 Launch Record Column Headers:")
print(column_names)


🚀 Launch Record Column Headers:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']


In [16]:
from bs4 import BeautifulSoup
import requests

# Fetch the HTML content from the Falcon 9 launch page
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find all tables on the page
tables = soup.find_all("table")

# Target the 3rd table (index 2)
target_table = tables[2]

# Locate the header row and extract all <th> elements
header_row = target_table.find("tr")
column_names = [th.get_text(strip=True) for th in header_row.find_all("th")]

# Display column names
print("🛰️ Launch Table Column Names:")
for col in column_names:
    print(f"• {col}")


🛰️ Launch Table Column Names:
• Flight No.
• Date andtime (UTC)
• Version,Booster[b]
• Launch site
• Payload[c]
• Payload mass
• Orbit
• Customer
• Launchoutcome
• Boosterlanding


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [17]:
# Target the third table (index 2)
target_table = tables[2]

# Find all <th> elements in the header row
header_row = target_table.find("tr")
th_elements = header_row.find_all("th")

# Apply extract_column_from_header() to each <th>
column_names = []
for th in th_elements:
    name = extract_column_from_header(th)
    if name:  # Skip empty or None results
        column_names.append(name)

# Display the cleaned column names
print("🧾 Extracted Column Names:")
for col in column_names:
    print(f"• {col}")


🧾 Extracted Column Names:
• Flight No.
• Date and time ( )
• Launch site
• Payload
• Payload mass
• Orbit
• Customer
• Launch outcome


Check the extracted column names


In [18]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [20]:
# Initialize dictionary with column names as keys and empty lists as values
launch_data_dict = {col_name: [] for col_name in column_names}

# Check the structure
print("🗂️ Initialized Dictionary:")
for key in launch_data_dict:
    print(f"{key}: {launch_data_dict[key]}")


🗂️ Initialized Dictionary:
Flight No.: []
Date and time ( ): []
Launch site: []
Payload: []
Payload mass: []
Orbit: []
Customer: []
Launch outcome: []


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


In [22]:
# Find all rows after the header
rows = target_table.find_all("tr")[1:]  # Skip header row

for row in rows:
    cells = row.find_all(["th", "td"])
    if len(cells) != len(column_names):
        continue  # Skip rows that don't match the expected number of columns

    for idx, cell in enumerate(cells):
        text = extract_column_from_header(cell).strip()
        launch_data_dict[column_names[idx]].append(text)


In [23]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [25]:
# TODO: Make sure target_table and column_names are already defined
rows = target_table.find_all("tr")[1:]  # Skip header

for row in rows:
    cells = row.find_all(["th", "td"])
    
    # Skip rows that don’t match the number of columns
    if len(cells) != len(column_names):
        continue  # TODO: Log or print skipped row for debugging if needed
    
    for idx, cell in enumerate(cells):
        # Clean cell text
        cell_text = extract_column_from_header(cell).strip()
        
        # TODO: Add custom parsing for dates or numbers if needed
        # Example: parse mass from '123 kg' to float
        
        # Append to the correct key in the dictionary
        launch_data_dict[column_names[idx]].append(cell_text)
df = pd.DataFrame(launch_data_dict)
print(df.head())


Empty DataFrame
Columns: [Flight No., Date and time ( ), Launch site, Payload, Payload mass, Orbit, Customer, Launch outcome]
Index: []


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [26]:
#df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

# Convert the populated dictionary into a DataFrame
df = pd.DataFrame(launch_data_dict)

# Optional: Preview the first few rows
print("🚀 Parsed Launch Data:")
print(df.head())


🚀 Parsed Launch Data:
Empty DataFrame
Columns: [Flight No., Date and time ( ), Launch site, Payload, Payload mass, Orbit, Customer, Launch outcome]
Index: []


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
